
# Análisis de clientes Haulmer : EDA parte dos

Por : Christian Vergara Retamal - Benjamín Sánchez Aliste

Los siguientes archivos fueron proporcionados por las diversas áreas que interactuan directa e indirectamente con los clientes de Haulmer. El notebook funciona como anezxo de los archivos subidos anteriormente al repositoria de análisis de los datos 




# Aspectos Computacionales

**Librerías utilizadas para el funcionamiento del notebook**

* numpy  versión 1.21.6 : Álgebra lineal y matemáticas
* pandas versión 1.3.5 : Manejo y análisis de estructuras de datos
* seaborn versión 0.11.2 : Visualización de gráficos
* matplotlib 3.2.2 : Gráficos
* Scikit-learn 1.0.2 : Machine Learning

# Preparación Ambiente de Trabajo

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import scipy.stats as stats 
import warnings 
warnings.filterwarnings('ignore')
from datetime import datetime
import statsmodels.api as sm
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [2]:
plt.style.use('seaborn-whitegrid')
plt.rcParams["figure.figsize"] = (10,6)
plt.rcParams["figure.dpi"] = 75 

# Archivo **Reporte Comercios s/actividad** 

In [4]:
df_sinact = pd.read_excel("/content/drive/MyDrive/Proyecto Innovación 1/Metodología/Datos/Reporte_de_comercios_solo_ocupar_los_NO_.xlsx")

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
df_sinact.columns

Index(['RUT', 'Nombre Legal', 'Nombre', 'Email', 'Nombre Plan',
       'Ha recibido pagos? (desde 2022-06-01)'],
      dtype='object')

In [7]:
print("Cantidad de registros : ",df_sinact.shape[0])
print("Cantidad de atributos : ",df_sinact.shape[1])

Cantidad de registros :  1190
Cantidad de atributos :  6


## Análisis exploratorio y preprocesamiento **Reporte Comercios**

In [8]:
df_sinact.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1190 entries, 0 to 1189
Data columns (total 6 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   RUT                                    1190 non-null   object
 1   Nombre Legal                           1190 non-null   object
 2   Nombre                                 1190 non-null   object
 3   Email                                  1190 non-null   object
 4   Nombre Plan                            1190 non-null   object
 5   Ha recibido pagos? (desde 2022-06-01)  1190 non-null   object
dtypes: object(6)
memory usage: 55.9+ KB


In [9]:
df_sinact.describe()

,RUT,Nombre Legal,Nombre,Email,Nombre Plan,Ha recibido pagos? (desde 2022-06-01)
count,1190,1190,1190,1190,1190,1190
unique,1186,1189,1189,1155,7,1
top,11111111-1,GERSON ARCADIO PULIDO RINCON,GERSON ARCADIO PULIDO RINCON,pymesimple@gmail.com,Plan Tuu,NO
freq,3,2,2,10,914,1190


### Columna RUT

In [10]:
df_sinact.RUT.value_counts()

11111111-1    3
19469561-6    2
25600822-K    2
77169667-8    1
15756911-2    1
             ..
9045455-2     1
77066762-3    1
11576701-1    1
76049706-1    1
10546401-0    1
Name: RUT, Length: 1186, dtype: int64

In [11]:
from itertools import cycle
 
def validarRut(rut):
	rut = rut.upper();
	rut = rut.replace("-","")
	rut = rut.replace(".","")
	aux = rut[:-1]
	dv = rut[-1:]
 
	revertido = map(int, reversed(str(aux)))
	factors = cycle(range(2,8))
	s = sum(d * f for d, f in zip(revertido,factors))
	res = (-s)%11
 
	if str(res) == dv:
		return True
	elif dv=="K" and res==10:
		return True
	else:
		return False

In [12]:
df_sinact["aux_rut"] = df_sinact["RUT"].apply(lambda x : validarRut(x))

In [13]:
df_sinact.aux_rut.value_counts()

True     1189
False       1
Name: aux_rut, dtype: int64

In [14]:
df_sinact = df_sinact[df_sinact.aux_rut == True]

In [15]:
df_sinact.drop(columns = ["aux_rut"],inplace=True)

In [16]:
df_sinact.drop_duplicates(inplace = True)

In [17]:
df_sinact = df_sinact[df_sinact.RUT != "11111111-1"]

In [18]:
df_sinact[df_sinact['Email'].str.contains('haulmer')]

,RUT,Nombre Legal,Nombre,Email,Nombre Plan,Ha recibido pagos? (desde 2022-06-01)
2,19469561-6,Alex QA,Alex QA,rocio.gomez@haulmer.com,Plan Tuu,NO
6,19469561-6,compu Alex SA,compu Alex SA,alexander.zuniga@haulmer.com,Plan Tuu,NO
23,76144941-9,FENG S.A,FENG S.A,chiyinfeng@haulmer.com,Plan Tuu,NO
916,26802941-9,ROGER EDUARDO GARCIA REYES,ROGER EDUARDO GARCIA REYES,roger.garcia@haulmer.com,Plan Tuu,NO


In [19]:
values = ["19469561-6", "26802941-9"]
df_sinact = df_sinact[df_sinact.RUT.isin(values) == False]

In [20]:
df_sinact[df_sinact['Email'].str.contains('haulmer')]

,RUT,Nombre Legal,Nombre,Email,Nombre Plan,Ha recibido pagos? (desde 2022-06-01)
23,76144941-9,FENG S.A,FENG S.A,chiyinfeng@haulmer.com,Plan Tuu,NO


### Nombre Plan

In [21]:
df_sinact["Nombre Plan"].value_counts()

Plan Tuu                   907
Plan Tuu [Partner]         118
Simple                      95
Simple [Partner]            30
Plan Tuu+                   24
OpenRetail Microempresa      5
Plan Tuu+ [Partner]          4
Name: Nombre Plan, dtype: int64

In [22]:
df_sinact.drop(columns = ["Ha recibido pagos? (desde 2022-06-01)"], inplace = True)

In [23]:
df_sinact.sample(5)

,RUT,Nombre Legal,Nombre,Email,Nombre Plan
678,77543875-4,STRONG MOTOS SPA,CARLA BEATRIZ QUINTANILLA,carla.bqb@gmail.com,Plan Tuu
473,4848312-7,MARIA LUISA OYARZO MANCILLA,MARIA LUISA OYARZO MANCILLA,gyflimitada2014@gmail.com,Plan Tuu
446,77473932-7,JOYAS DEL ITALIANO SPA,Rodrigo Vallebona,vallebonarodrigo@gmail.com,Plan Tuu
777,8723790-7,Alicia Reyes Castillo,JORGE ULLOA PICERO,julloa.buses@gmail.com,Plan Tuu
325,77404705-0,JJELECTRONIC SPA,JJELECTRONIC SPA,jazminv.riquelme.82@gmail.com,Plan Tuu


In [24]:
categorical_cols = ['Nombre Plan']
lencoder = LabelEncoder()
for col in categorical_cols :
  df_sinact[col] = df_sinact[[col]].apply(lencoder.fit_transform)

In [25]:
df_sinact.sample(5)

,RUT,Nombre Legal,Nombre,Email,Nombre Plan
34,77430115-1,MULTYMASITAS SPA,MARGARITA ANDREA PEREIRE,maggyta_25@hotmail.com,1
168,76933349-5,COMERCIAL VIENTO DE OTOÑO LIMITADA,COMERCIAL VIENTO DE OTOÑO LIMITADA,C.VIENTODEOTONO@GMAIL.COM,1
291,77000448-9,AUTOMOTORA HIDALGO SPA,Roberto Aliro Hidalgo,robertoautomotriz1983@gmail.com,1
526,77463728-1,QUIRAH SPA,QUIRAH SPA,contacto@masamorbakery.cl,1
352,14620514-3,ALICIA ALEJANDRA MIRA SILVA,ALICIA ALEJANDRA MIRA SILVA,alicia.mira.silva@gmail.com,1


----

# Archivo **Certificaciones**

In [26]:
df_certificaciones = pd.read_excel("/content/drive/MyDrive/Proyecto Innovación 1/Metodología/Datos/Certificaciones.xlsx")

In [27]:
df_certificaciones.columns

Index(['Clave', 'Resumen', 'Responsable', 'Estado', 'Creada', 'Actualizada',
       'RUT Empresa'],
      dtype='object')

In [28]:
print("Cantidad de registros : ",df_certificaciones.shape[0])
print("Cantidad de atributos : ",df_certificaciones.shape[1])

Cantidad de registros :  33815
Cantidad de atributos :  7


## Análisis exploratorio y preprocesamiento **Certificaciones**

In [29]:
df_certificaciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33815 entries, 0 to 33814
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Clave        33815 non-null  object        
 1   Resumen      33815 non-null  object        
 2   Responsable  3303 non-null   object        
 3   Estado       33815 non-null  object        
 4   Creada       33815 non-null  datetime64[ns]
 5   Actualizada  33815 non-null  datetime64[ns]
 6   RUT Empresa  27102 non-null  object        
dtypes: datetime64[ns](2), object(5)
memory usage: 1.8+ MB


In [30]:
df_certificaciones.describe()

,Clave,Resumen,Responsable,Estado,Creada,Actualizada,RUT Empresa
count,33815,33815,3303,33815,33815,33815,27102
unique,33815,33283,18,36,33656,26580,14661
top,CO-34266,: Certificación DTE: LIVEHOST SPA,Jorge Calquín,Finalizada,2022-06-07 05:49:51,2020-12-30 14:15:25,76353391-3
freq,1,47,1073,16383,3,16,89
first,NaN,NaN,NaN,NaN,2018-03-15 06:15:49,2020-12-30 14:02:17,NaN
last,NaN,NaN,NaN,NaN,2022-06-13 10:48:17,2022-06-13 11:35:19,NaN


In [31]:
df_certificaciones.describe(include="object")

,Clave,Resumen,Responsable,Estado,RUT Empresa
count,33815,33815,3303,33815,27102
unique,33815,33283,18,36,14661
top,CO-34266,: Certificación DTE: LIVEHOST SPA,Jorge Calquín,Finalizada,76353391-3
freq,1,47,1073,16383,89


In [32]:
df_certificaciones.sample(3)

,Clave,Resumen,Responsable,Estado,Creada,Actualizada,RUT Empresa
7576,CO-26689,PROD_WEB: Certificación Boleta: RACHEL ODHALY ...,NaN,Listo para activar,2021-08-04 08:36:14,2021-08-04 08:46:48,20033214-8
16612,CO-17647,PROD_WEB: Certificación Inicial: silvia de car...,NaN,Terminado,2021-02-09 14:19:51,2021-02-21 11:49:17,4990805-9
28837,CO-5376,PROD_WEB: Certificación Boleta: SUPER ALIMENTO...,NaN,Finalizada,2020-06-01 13:18:39,2020-12-30 15:15:28,NaN


----

# Archivo **Consolidados Pagos TUU**

In [33]:
df_consolidados = pd.read_excel("/content/drive/MyDrive/Proyecto Innovación 1/Metodología/Datos/Consolidado TUU Pagos 2022.xlsx")

In [34]:
print("Cantidad de registros : ",df_consolidados.shape[0])
print("Cantidad de atributos : ",df_consolidados.shape[1])

Cantidad de registros :  17319
Cantidad de atributos :  28


## Análisis exploratorio y preprocesamiento **Consolidados Pagos TUU**

In [35]:
df_consolidados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17319 entries, 0 to 17318
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   RUT                      17319 non-null  object        
 1   Razón social             17319 non-null  object        
 2   Código de comercio       17319 non-null  int64         
 3   Código SII               17319 non-null  float64       
 4   Porcentaje comision      17319 non-null  float64       
 5   Comision fija            17319 non-null  float64       
 6   Cantidad TXs             17319 non-null  float64       
 7   Monto Total              17319 non-null  float64       
 8   Comision Neta            17319 non-null  float64       
 9   Comision IVA             17319 non-null  float64       
 10  Tipo                     17319 non-null  object        
 11  Período                  17319 non-null  datetime64[ns]
 12  Porcentaje comision 2    17319 n

In [36]:
df_consolidados.describe()

,Código de comercio,Código SII,Porcentaje comision,Comision fija,Cantidad TXs,Monto Total,Comision Neta,Comision IVA,Porcentaje comision 2,MDR Abril Débito,...,Costo tx Abril,Margen tx Abril,MDR Mayo REAL,MDR Mayo Supuesto,Costo tx Mayo REAL,Margen tx Mayo REAL,Costo tx Mayo Supuesto,Margen tx Mayo Supuesto,Tarifa Partner,Margen Partner
count,1.731900e+04,17319.000000,17319.000000,17319.0,17319.000000,1.731900e+04,17319.000000,17319.000000,17319.000000,17319.000000,...,17319.000000,17319.000000,14821.000000,14821.000000,14821.000000,14821.000000,14821.000000,17319.000000,17319.000000,17319.000000
mean,5.970395e+11,500666.757838,1.657170,0.0,126.724291,1.433976e+06,19524.408684,3710.670131,0.016572,0.012083,...,18793.937244,730.471440,0.014151,0.014151,18654.139746,2725.815116,18654.139746,2332.658111,0.016424,228.403173
std,4.086982e+05,184546.689871,0.553607,0.0,282.112957,3.051467e+06,40812.801621,7755.479639,0.005536,0.001071,...,39133.343188,6978.346002,0.004417,0.004417,37143.484334,8867.337013,37143.484334,8258.634385,0.005470,3994.321917
min,5.970392e+11,11306.000000,0.000000,0.0,1.000000,1.000000e+02,0.000000,0.000000,0.000000,0.005600,...,0.560000,-247634.662500,0.005100,0.005100,0.900000,-207830.951100,0.900000,-207830.951100,0.001300,-123458.715000
25%,5.970393e+11,472101.000000,1.290000,0.0,4.000000,6.799500e+04,1217.000000,231.000000,0.012900,0.011700,...,1161.020000,0.913000,0.010700,0.010700,1488.081000,128.000000,1488.081000,16.580000,0.012900,0.000000
50%,5.970393e+11,474100.000000,1.290000,0.0,22.000000,3.540000e+05,5442.000000,1035.000000,0.012900,0.012000,...,5227.250000,120.100000,0.011300,0.011300,5894.200000,718.391000,5894.200000,447.800000,0.012900,0.000000
75%,5.970393e+11,561000.000000,2.250000,0.0,113.000000,1.426310e+06,19964.000000,3800.000000,0.022500,0.012700,...,19182.884500,781.000000,0.019300,0.019300,19732.812100,2852.834000,19732.812100,2263.141000,0.022500,0.000000
max,5.970406e+11,960909.000000,2.900000,0.0,5221.000000,6.179230e+07,797164.000000,151416.000000,0.029000,0.025000,...,722969.874900,154081.256000,0.035000,0.035000,698252.956100,211239.085000,698252.956100,211239.085000,0.025100,151592.503000


In [37]:
df_consolidados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17319 entries, 0 to 17318
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   RUT                      17319 non-null  object        
 1   Razón social             17319 non-null  object        
 2   Código de comercio       17319 non-null  int64         
 3   Código SII               17319 non-null  float64       
 4   Porcentaje comision      17319 non-null  float64       
 5   Comision fija            17319 non-null  float64       
 6   Cantidad TXs             17319 non-null  float64       
 7   Monto Total              17319 non-null  float64       
 8   Comision Neta            17319 non-null  float64       
 9   Comision IVA             17319 non-null  float64       
 10  Tipo                     17319 non-null  object        
 11  Período                  17319 non-null  datetime64[ns]
 12  Porcentaje comision 2    17319 n

In [38]:
df_consolidados.describe()

,Código de comercio,Código SII,Porcentaje comision,Comision fija,Cantidad TXs,Monto Total,Comision Neta,Comision IVA,Porcentaje comision 2,MDR Abril Débito,...,Costo tx Abril,Margen tx Abril,MDR Mayo REAL,MDR Mayo Supuesto,Costo tx Mayo REAL,Margen tx Mayo REAL,Costo tx Mayo Supuesto,Margen tx Mayo Supuesto,Tarifa Partner,Margen Partner
count,1.731900e+04,17319.000000,17319.000000,17319.0,17319.000000,1.731900e+04,17319.000000,17319.000000,17319.000000,17319.000000,...,17319.000000,17319.000000,14821.000000,14821.000000,14821.000000,14821.000000,14821.000000,17319.000000,17319.000000,17319.000000
mean,5.970395e+11,500666.757838,1.657170,0.0,126.724291,1.433976e+06,19524.408684,3710.670131,0.016572,0.012083,...,18793.937244,730.471440,0.014151,0.014151,18654.139746,2725.815116,18654.139746,2332.658111,0.016424,228.403173
std,4.086982e+05,184546.689871,0.553607,0.0,282.112957,3.051467e+06,40812.801621,7755.479639,0.005536,0.001071,...,39133.343188,6978.346002,0.004417,0.004417,37143.484334,8867.337013,37143.484334,8258.634385,0.005470,3994.321917
min,5.970392e+11,11306.000000,0.000000,0.0,1.000000,1.000000e+02,0.000000,0.000000,0.000000,0.005600,...,0.560000,-247634.662500,0.005100,0.005100,0.900000,-207830.951100,0.900000,-207830.951100,0.001300,-123458.715000
25%,5.970393e+11,472101.000000,1.290000,0.0,4.000000,6.799500e+04,1217.000000,231.000000,0.012900,0.011700,...,1161.020000,0.913000,0.010700,0.010700,1488.081000,128.000000,1488.081000,16.580000,0.012900,0.000000
50%,5.970393e+11,474100.000000,1.290000,0.0,22.000000,3.540000e+05,5442.000000,1035.000000,0.012900,0.012000,...,5227.250000,120.100000,0.011300,0.011300,5894.200000,718.391000,5894.200000,447.800000,0.012900,0.000000
75%,5.970393e+11,561000.000000,2.250000,0.0,113.000000,1.426310e+06,19964.000000,3800.000000,0.022500,0.012700,...,19182.884500,781.000000,0.019300,0.019300,19732.812100,2852.834000,19732.812100,2263.141000,0.022500,0.000000
max,5.970406e+11,960909.000000,2.900000,0.0,5221.000000,6.179230e+07,797164.000000,151416.000000,0.029000,0.025000,...,722969.874900,154081.256000,0.035000,0.035000,698252.956100,211239.085000,698252.956100,211239.085000,0.025100,151592.503000


In [39]:
df_consolidados.describe(include = "object")

,RUT,Razón social,Tipo,MCC,Partner,Categoría Partner
count,17319,17319,17319,16900,17319,17309
unique,2955,3017,2,86,31,5
top,74828100-2,FUND JARDIN BOTANICO NACIONAL DE VINA DEL MAR,DEBITO,SUPERMERCADOS,No Partner,No Partner
freq,60,60,10178,3334,15568,15568


In [40]:
df_consolidados["aux_rut"] = df_consolidados["RUT"].apply(lambda x : validarRut(x))

In [41]:
df_consolidados["aux_rut"].value_counts()

True    17319
Name: aux_rut, dtype: int64

In [42]:
df_consolidados["RUT"].value_counts()

74828100-2    60
76795561-8    46
76544003-3    39
76938573-8    38
78067040-1    38
              ..
76513949-K     1
76510731-8     1
76505877-5     1
10134744-3     1
77525046-1     1
Name: RUT, Length: 2955, dtype: int64

In [43]:
df_consolidados.RUT.nunique()

2955

In [44]:
df_consolidados.Período.value_counts()

2022-05-01    4897
2022-04-01    4079
2022-03-01    3366
2022-02-01    2731
2022-01-01    2246
Name: Período, dtype: int64

In [45]:
df_consolidados.columns

Index(['RUT', 'Razón social', 'Código de comercio', 'Código SII',
       'Porcentaje comision', 'Comision fija', 'Cantidad TXs', 'Monto Total',
       'Comision Neta', 'Comision IVA', 'Tipo', 'Período',
       'Porcentaje comision 2', 'MDR Abril Débito', 'MDR Abril Crédito',
       'Costo tx Abril', 'Margen tx Abril', 'MDR Mayo REAL',
       'MDR Mayo Supuesto', 'Costo tx Mayo REAL', 'Margen tx Mayo REAL',
       'Costo tx Mayo Supuesto', 'Margen tx Mayo Supuesto', 'MCC', 'Partner',
       'Categoría Partner', 'Tarifa Partner', 'Margen Partner', 'aux_rut'],
      dtype='object')

In [46]:
df_consolidados_aux = df_consolidados.copy()
df_consolidados_aux.drop(columns = ["Código de comercio", "Período", "Porcentaje comision 2", 'MDR Abril Débito', 'MDR Abril Crédito',
       'Costo tx Abril', 'Margen tx Abril', 'MDR Mayo REAL',
       'MDR Mayo Supuesto', 'Costo tx Mayo REAL', 'Margen tx Mayo REAL',
       'Costo tx Mayo Supuesto', 'Margen tx Mayo Supuesto', "Tarifa Partner", "Margen Partner", "aux_rut"], inplace = True)

In [47]:
df_consolidados_aux.sample(2)

,RUT,Razón social,Código SII,Porcentaje comision,Comision fija,Cantidad TXs,Monto Total,Comision Neta,Comision IVA,Tipo,MCC,Partner,Categoría Partner
163,12946162-4,Mirta Idolia Cordero Rivera,472103.0,1.29,0.0,121.0,428780.0,5536.0,1051.0,DEBITO,OTRAS TIENDAS DE ALIMENTOS,No Partner,No Partner
1918,77113034-8,COMERCIAL MC SPA,472101.0,2.32,0.0,34.0,167500.0,3886.0,738.0,CREDITO,SUPERMERCADOS,No Partner,No Partner


In [48]:
df_consolidados_aux.rename(columns = {"Razón social":"razonSocial", "Código SII":"codSII", "Porcentaje comision":"porcentajeComision", "Comision fija":"comisionFija",
                                  "Cantidad TXs":"cantidadTransacciones", "Monto Total":"montoTotal", "Comision Neta":"comisionNeta",
                                  "Comision IVA":"comisionIVA", "Categoría Partner":"categoriaPartner"}, inplace = True)

In [49]:
df_consolidados_aux.sample(2)

,RUT,razonSocial,codSII,porcentajeComision,comisionFija,cantidadTransacciones,montoTotal,comisionNeta,comisionIVA,Tipo,MCC,Partner,categoriaPartner
2969,76422106-0,CENTRO DEPORTIVO BAEZA Y BAEZA SPORTING CLUB L...,931109.0,1.31,0.0,38.0,621000.0,8145.0,1541.0,DEBITO,CAMPOS DE ATLETISMO Y DEPORTES,No Partner,No Partner
627,76557454-4,DISTRIBUIDORA Y COMERCIALIZADORA CHRISTIAN ALV...,463019.0,1.17,0.0,1710.0,15857889.0,185500.0,35262.0,DEBITO,OTRAS TIENDAS DE ALIMENTOS,No Partner,No Partner


In [50]:
df_consolidados_aux.shape

(17319, 13)

In [51]:
df_consolidados_aux.codSII.value_counts()

472101.0    3105
561000.0    1971
107100.0     632
472109.0     370
960200.0     370
            ... 
476102.0       1
949909.0       1
12900.0        1
742001.0       1
370000.0       1
Name: codSII, Length: 249, dtype: int64

In [52]:
def getfreqcred(rut):
  df_rut = df_consolidados_aux[df_consolidados_aux["RUT"] == rut]
  return df_rut[df_rut["Tipo"] == "CREDITO"].shape[0]

In [53]:
def getfreqdeb(rut):
  df_rut = df_consolidados_aux[df_consolidados_aux["RUT"] == rut]
  return df_rut[df_rut["Tipo"] == "DEBITO"].shape[0]

In [54]:
df_montoTotal = df_consolidados_aux.groupby(['RUT'])['montoTotal'].sum().to_frame().reset_index()
df_cantTxs = df_consolidados_aux.groupby(['RUT'])['cantidadTransacciones'].sum().to_frame().reset_index()
df_neta = df_consolidados_aux.groupby(['RUT'])['comisionNeta'].sum().to_frame().reset_index()
df_iva = df_consolidados_aux.groupby(['RUT'])['comisionIVA'].sum().to_frame().reset_index()
df_porcentajeComision = df_consolidados_aux.groupby(['RUT'])["porcentajeComision"].mean().to_frame()

In [55]:
categorical_cols = ["codSII", 'MCC', 'Partner', 'categoriaPartner']
lencoder = LabelEncoder()
for col in categorical_cols :
  df_consolidados_aux[col] = df_consolidados_aux[[col]].apply(lencoder.fit_transform)

In [56]:
df_consolidados_aux.drop(columns = ["montoTotal", "cantidadTransacciones", "comisionNeta", "comisionIVA", "porcentajeComision", "comisionFija", "razonSocial"], inplace = True)

In [57]:
df_consolidados_aux

,RUT,codSII,Tipo,MCC,Partner,categoriaPartner
0,10018933-K,23,DEBITO,48,0,3
1,10037348-3,116,DEBITO,11,0,3
2,10037348-3,116,DEBITO,11,0,3
3,10049357-8,107,DEBITO,74,0,3
4,10054812-7,173,DEBITO,66,24,2
...,...,...,...,...,...,...
17314,9985572-K,18,DEBITO,57,28,4
17315,9985572-K,18,DEBITO,57,28,4
17316,9988020-1,18,DEBITO,57,0,3
17317,9999023-6,112,CREDITO,48,0,3


In [58]:
df_consolidados_aux["debito"] = df_consolidados_aux["RUT"].apply(lambda x : getfreqdeb(x))
df_consolidados_aux["credito"] = df_consolidados_aux["RUT"].apply(lambda x : getfreqcred(x))

In [59]:
df_consolidados_aux.drop(columns = ["Tipo"], inplace = True)

In [60]:
df_debito = df_consolidados_aux.groupby(['RUT'])["debito"].mean().to_frame()
df_credito = df_consolidados_aux.groupby(['RUT'])["credito"].mean().to_frame()

In [62]:
df_aux = df_consolidados_aux.copy()
df_aux.drop_duplicates(inplace = True)

In [63]:
df_aux

,RUT,codSII,MCC,Partner,categoriaPartner,debito,credito
0,10018933-K,23,48,0,3,5,4
1,10037348-3,116,11,0,3,8,6
3,10049357-8,107,74,0,3,5,5
4,10054812-7,173,66,24,2,3,0
5,10056607-9,110,74,0,3,5,4
...,...,...,...,...,...,...,...
17287,9751797-5,110,74,17,2,1,0
17294,9789513-9,114,57,0,3,1,0
17302,9836941-4,110,74,0,3,1,0
17305,9846371-2,111,13,29,4,1,1


*Se opta por eliminar los rut duplicados para la primera iteración, posteriormente se espera buscar una forma de tratar aquellos comercios que contengan más de un MCC / código servicio impuestos internos*

In [64]:
df_aux.drop_duplicates(subset = "RUT", keep="first", inplace = True)

In [65]:
df_aux

,RUT,codSII,MCC,Partner,categoriaPartner,debito,credito
0,10018933-K,23,48,0,3,5,4
1,10037348-3,116,11,0,3,8,6
3,10049357-8,107,74,0,3,5,5
4,10054812-7,173,66,24,2,3,0
5,10056607-9,110,74,0,3,5,4
...,...,...,...,...,...,...,...
17287,9751797-5,110,74,17,2,1,0
17294,9789513-9,114,57,0,3,1,0
17302,9836941-4,110,74,0,3,1,0
17305,9846371-2,111,13,29,4,1,1


In [66]:
data_frames = [df_montoTotal, df_cantTxs, df_neta, df_iva, df_porcentajeComision, df_debito, df_credito, df_aux]

In [67]:
from functools import reduce
df_consolidados_merge = reduce(lambda  left,right: pd.merge(left,right,on=['RUT']), data_frames)

In [68]:
df_consolidados_merge['montoTotal'] = df_consolidados_merge['montoTotal'].astype(int)
df_consolidados_merge['cantidadTransacciones'] = df_consolidados_merge['cantidadTransacciones'].astype(int)
df_consolidados_merge['comisionNeta'] = df_consolidados_merge['comisionNeta'].astype(int)
df_consolidados_merge['comisionIVA'] = df_consolidados_merge['comisionIVA'].astype(int)

In [69]:
df_consolidados_merge.sample(3)

,RUT,montoTotal,cantidadTransacciones,comisionNeta,comisionIVA,porcentajeComision,debito_x,credito_x,codSII,MCC,Partner,categoriaPartner,debito_y,credito_y
1045,6580318-6,1000,1,13,2,1.2900,1.0,0.0,110,74,0,3,1,0
2380,77487781-9,54900,14,694,130,1.2700,2.0,0.0,195,52,28,4,2,0
37,10291334-5,2610174,726,33993,6480,1.5475,3.0,1.0,110,74,0,3,3,1


In [70]:
df_consolidados_merge["porcentajeComision"] = df_consolidados_merge["porcentajeComision"].round(decimals = 2)

In [71]:
df_consolidados_merge

,RUT,montoTotal,cantidadTransacciones,comisionNeta,comisionIVA,porcentajeComision,debito_x,credito_x,codSII,MCC,Partner,categoriaPartner,debito_y,credito_y
0,10006728-5,209000,8,2697,513,1.29,1.0,0.0,142,3,0,3,1,0
1,10007508-3,300,2,4,1,1.29,1.0,0.0,111,13,0,3,1,0
2,10011781-9,8410468,387,118884,22577,1.88,4.0,3.0,111,13,11,4,4,3
3,10018933-K,4526000,178,55777,10587,1.63,5.0,4.0,23,48,0,3,5,4
4,10024696-1,927100,89,13335,2536,1.90,2.0,2.0,109,86,0,3,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2950,9974219-4,11297545,1194,141104,26821,1.69,3.0,3.0,154,48,0,3,3,3
2951,9979750-9,1270180,515,13431,2583,1.50,5.0,5.0,114,57,0,3,5,5
2952,9985572-K,891200,124,10065,1916,1.13,2.0,0.0,18,57,28,4,2,0
2953,9988020-1,27783870,840,319914,60777,1.45,5.0,4.0,18,57,0,3,5,4


In [72]:
df_consolidados_merge.drop(columns = ["debito_x", "credito_x"], inplace=True)

In [73]:
df_consolidados_merge.rename(columns = {"credito_y":"credito", "debito_y":"debito"}, inplace = True)

In [74]:
df_consolidados_merge

,RUT,montoTotal,cantidadTransacciones,comisionNeta,comisionIVA,porcentajeComision,codSII,MCC,Partner,categoriaPartner,debito,credito
0,10006728-5,209000,8,2697,513,1.29,142,3,0,3,1,0
1,10007508-3,300,2,4,1,1.29,111,13,0,3,1,0
2,10011781-9,8410468,387,118884,22577,1.88,111,13,11,4,4,3
3,10018933-K,4526000,178,55777,10587,1.63,23,48,0,3,5,4
4,10024696-1,927100,89,13335,2536,1.90,109,86,0,3,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...
2950,9974219-4,11297545,1194,141104,26821,1.69,154,48,0,3,3,3
2951,9979750-9,1270180,515,13431,2583,1.50,114,57,0,3,5,5
2952,9985572-K,891200,124,10065,1916,1.13,18,57,28,4,2,0
2953,9988020-1,27783870,840,319914,60777,1.45,18,57,0,3,5,4


----

# Archivo **Facturación y pagos clientes TUU**

In [75]:
df_facturacion_sheetcl = pd.read_excel("/content/drive/MyDrive/Proyecto Innovación 1/Metodología/Datos/Facturación y pagos clientes TUU.xlsx", "Cliente Tuu")

In [76]:
df_facturacion_sheetcl.columns

Index(['Rut', 'Rut dv', 'Rut sdv', 'Email', 'Whmcs', 'Workspace', 'Plan',
       'Organizacion', 'Facturación', 'Pagos', 'F2', 'P2'],
      dtype='object')

In [77]:
print("Cantidad de registros : ",df_facturacion_sheetcl.shape[0])
print("Cantidad de atributos : ",df_facturacion_sheetcl.shape[1])

Cantidad de registros :  4010
Cantidad de atributos :  12


In [79]:
df_facturacion_sheetpag = pd.read_excel("/content/drive/MyDrive/Proyecto Innovación 1/Metodología/Datos/Pagos.xlsx")

In [80]:
print("Cantidad de registros : ",df_facturacion_sheetpag.shape[0])
print("Cantidad de atributos : ",df_facturacion_sheetpag.shape[1])

Cantidad de registros :  17539
Cantidad de atributos :  5


## Análisis exploratorio y preprocesamiento **Facturación y pagos clientes TUU**

In [81]:
df_facturacion_sheetcl.sample(3)

,Rut,Rut dv,Rut sdv,Email,Whmcs,Workspace,Plan,Organizacion,Facturación,Pagos,F2,P2
1213,146832377,14683237-7,14683237,wilmanmendez20@gmail.com,99259.0,16812.0,Plan Tuu,Confecciones Beni,NaN,Realiza Pagos,e. 10MM,c. 1MM
3780,6146459K,6146459-K,6146459,castillo.maribel@gmail.com,102200.0,17474.0,Plan Tuu,Deyse Ardiles Contreras,NaN,NaN,a. 0MM,a. 0MM
3229,774765999,77476599-9,77476599,karlita.vallejos@gmail.com,87307.0,15297.0,Plan Tuu,Bitter Dolls Spa,NaN,Realiza Pagos,d. 3MM,d. 3MM


In [82]:
df_facturacion_sheetpag.sample(3)

,RUT,Tipo,Periodo,CantidadTXs,MontoTotal
13267,13783568-1,Crédito,2022-05-01,1.0,7000.0
16662,77492483-3,Débito,2022-05-01,43.0,524200.0
10515,12592066-7,Débito,2022-04-01,575.0,3896140.0


----

# Archivo **Incidencias**

In [83]:
df_incidencias = pd.read_excel("/content/drive/MyDrive/Proyecto Innovación 1/Metodología/Datos/Incidencias Christian RPA- HES.xlsx", "IncidenciasHES")

In [84]:
df_incidencias.columns

Index(['Clave', 'Resumen', 'Responsable', 'Informador', 'Estado', 'Creada',
       'Actualizada', 'Cuenta correo origen (Envía)',
       'Cuentas de Email con Problemas', 'Descripción', 'DTE con Problemas',
       'Email asociado a la compra', 'Email Cliente', 'ID Dispositivo',
       'Nombre Cliente', 'RUT asociado a la firma', 'RUT Empresa', 'Telefono',
       'Periodo'],
      dtype='object')

In [85]:
print("Cantidad de registros : ",df_incidencias.shape[0])
print("Cantidad de atributos : ",df_incidencias.shape[1])

Cantidad de registros :  12471
Cantidad de atributos :  19


## Análisis exploratorio y preprocesamiento **Incidencias**

In [86]:
df_incidencias.sample(3)

,Clave,Resumen,Responsable,Informador,Estado,Creada,Actualizada,Cuenta correo origen (Envía),Cuentas de Email con Problemas,Descripción,DTE con Problemas,Email asociado a la compra,Email Cliente,ID Dispositivo,Nombre Cliente,RUT asociado a la firma,RUT Empresa,Telefono,Periodo
1223,HES-11344,[WorkSpace] gummygummylovers@gmail.com,Jorge Caceres,Jaspe Boggio,Finalizada,2022-04-19 15:27:23,2022-04-19 17:28:43,NaN,NaN,Cliente me solicita cambiar el correo owner de...,NaN,NaN,gummygummylovers@gmail.com,NaN,Stephany Muñoz,NaN,NaN,NaN,2022-04-01
1491,HES-11070,[openfactura] 76809261-3,Fabian Rubio,Denis Salinas,Finalizada,2022-04-05 10:22:12,2022-04-05 17:36:09,NaN,NaN,Cliente solicita cambiar correo admin porque y...,NaN,NaN,contacto@alambiqueschile.cl,NaN,José Barra,NaN,76809261-3,+56977957154,2022-04-01
9015,HES-3515,[Hosty] mysabogados.cl,Fabian Rubio,Anibal Torres,Cerrada,2021-05-19 17:33:18,2021-05-28 15:23:00,NaN,NaN,Cliente indica que todos los correos que van h...,NaN,NaN,mcossiob@gmail.com,NaN,Macarena Cossio,NaN,NaN,+56998250887,2021-05-01


In [87]:
df_incidencias["RUT Empresa"].value_counts()

5213839-6     16
76975135-1    16
69040700-0    16
76540930-6    13
76974235-2    12
              ..
16040374-9     1
6750600-6      1
18252990-7     1
10691348-k     1
76905828       1
Name: RUT Empresa, Length: 5314, dtype: int64

In [107]:
df_incidencias = df_incidencias[df_incidencias['RUT Empresa'].notna()]

In [111]:
df_incidencias["RUT Empresa"].isna().sum()

0

In [118]:
df_incidencias

,Clave,Resumen,Responsable,Informador,Estado,Creada,Actualizada,Cuenta correo origen (Envía),Cuentas de Email con Problemas,Descripción,DTE con Problemas,Email asociado a la compra,Email Cliente,ID Dispositivo,Nombre Cliente,RUT asociado a la firma,RUT Empresa,Telefono,Periodo
0,HES-12573,[CambioCorreoEspacio] RUT: 12731763-1,NaN,Claudio Ignacio Arias Bastias,Tareas por hacer,2022-06-13 13:47:28,2022-06-13 13:52:19,NaN,NaN,Cliente solicita cambiar el correo vinculado a...,NaN,NaN,finanzas@empresasrvc.cl,NaN,Ricardo Vidal,NaN,12731763-1,941709754,2022-06-01
1,HES-12572,[OpenFactura] 77159914-1,Jorge Caceres,Carlos Rojas,In Review,2022-06-13 13:40:46,2022-06-13 14:05:04,NaN,NaN,Cliente con servicio suspendido (no desea reno...,DTE 39 Boleta Electrónica,NaN,enixtecnologiaycomputacion@gmail.com,NaN,Alberto Rivas,NaN,77159914-1,+56 9 65634449,2022-06-01
3,HES-12570,[SIMPLE] 76869731-0,Jorge Caceres,Gustavo Moya,In Review,2022-06-13 12:55:21,2022-06-13 13:50:04,NaN,NaN,"Cliente emite desde el pos, el día 28 de mayo ...",NaN,NaN,contacto@opticasclubvision.cl,NaN,Chrtian Silva Escobar,NaN,76869731-0,+56 9 71067982,2022-06-01
4,HES-12569,[TUU] 16429541-9,Fernanda Parras,Rodrigo Gilles,Tareas por hacer,2022-06-13 12:11:57,2022-06-13 12:51:55,NaN,NaN,Cliente indica que desde el día miércoles 8 de...,NaN,NaN,jmanuel.alvarez.osorio@gmail.com,NaN,MANUEL ALEJANDRO ALVAREZ JARA,NaN,16429541-9,+569 8124 8918,2022-06-01
6,HES-12567,[TUU] 11891533-K,NaN,Rodrigo Gilles,Urgente,2022-06-13 11:36:57,2022-06-13 11:37:01,NaN,NaN,Cliente indica que todos estos meses a pagados...,NaN,NaN,salvadorvale71@gmail.com,NaN,SALVADOR SEGUNDO VALENZUELA SALAS,NaN,11891533-K,+56945469089,2022-06-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12464,HES-59,[openfactura] 77020975-7,Juan Gutierrez,Gonzalo Labarca,Finalizada,2020-12-02 15:35:23,2021-01-06 18:08:38,NaN,NaN,Cliente indica que la boleta emitida el dia 29...,NaN,NaN,javier.stevens.c@gmail.com,3522,Javier Stevens,NaN,77020975-7,+569 99989197,NaT
12465,HES-58,[opefactura ]76743832-K,Fabian Rubio,Pedro Saavedra,Finalizada,2020-12-02 14:06:04,2021-01-06 18:08:37,NaN,NaN,el cliente señala que las facturas de compra s...,DTE 33 Factura Electrónica,NaN,diego.naranjo@exporteas.cl,NaN,diego diego,NaN,76743832-K,+56.976176077,NaT
12466,HES-57,[opefactura ]76860171-2,Fabian Rubio,Pedro Saavedra,Finalizada,2020-12-02 14:00:59,2021-01-06 18:08:41,NaN,NaN,cliente señala que las facturas de los proveed...,DTE 46 Factura de Compra,NaN,rivero.liliana@gmail.com,NaN,Rina RIvero,NaN,76860171-2,998204511,NaT
12467,HES-56,[openfactura] 7996710-6,Fabian Rubio,Pedro Saavedra,Finalizada,2020-12-02 13:56:22,2021-01-06 18:08:39,NaN,NaN,cliente indica que sus documentos dte 39 no se...,DTE 39 Boleta Electrónica,NaN,\toscar.figueroa@bmxshop.cl,NaN,OSCAR FIGUEROA,NaN,7996710-6,+56.976991334,NaT


----

# Archivo **Planes clientes directos**

In [92]:
df_directos = pd.read_excel("/content/drive/MyDrive/Proyecto Innovación 1/Metodología/Datos/Planes clientes directos.xlsx")

In [93]:
df_directos.columns

Index(['EMAIL', 'WHMCS', 'WORKSPACE', 'RUT', 'Plan Tuu', 'ORGANIZACION'], dtype='object')

In [94]:
print("Cantidad de registros : ",df_directos.shape[0])
print("Cantidad de atributos : ",df_directos.shape[1])

Cantidad de registros :  13571
Cantidad de atributos :  6


## Análisis exploratorio y preprocesamiento **Planes clientes directos**

In [95]:
df_directos.sample(3)

,EMAIL,WHMCS,WORKSPACE,RUT,Plan Tuu,ORGANIZACION
28,nicolasfigueroam@gmail.com,38753.0,80.0,184988798.0,Simple,carnes figueroa
10153,abdielaviles.chacon@gmail.com,95965.0,16525.0,155230479.0,Plan Tuu,Varinia tarifeño
1710,asesoriasof@gmail.com,50464.0,2832.0,767378335.0,Plan Tuu,Productos Agrícolas Verónica Fernandez Contrer...


----

# Archivo **Servicios Haulmer**

In [96]:
df_sshaulmer = pd.read_excel("/content/drive/MyDrive/Proyecto Innovación 1/Metodología/Datos/Servicios Haulmer (churn).xlsx")

In [97]:
df_sshaulmer.columns

Index(['Usuario', 'Servicio', 'Partner', 'Empresa', 'Categoria', 'Producto',
       'Monto Servicio', 'Registro', 'Periodo Registro', 'PeriodoTérmino',
       'Estado', 'Pagadas', 'Sin Pagar'],
      dtype='object')

In [98]:
print("Cantidad de registros : ",df_sshaulmer.shape[0])
print("Cantidad de atributos : ",df_sshaulmer.shape[1])

Cantidad de registros :  22129
Cantidad de atributos :  13


## Análisis exploratorio y preprocesamiento **Servicios Haulmer**

In [99]:
df_sshaulmer.sample(3)

,Usuario,Servicio,Partner,Empresa,Categoria,Producto,Monto Servicio,Registro,Periodo Registro,PeriodoTérmino,Estado,Pagadas,Sin Pagar
13838,30017.0,65708.0,NaN,Haulmer,Tuu,Partner | Tuu,0.0,2021-03-01,2021-03-01,2023-02-01 00:00:00,Active,2.0,0.0
5659,18841.0,34898.0,NaN,Haulmer,OpenFactura,OpenFactura Microempresa,120000.0,2020-08-26,2020-08-01,2020-09-01 00:00:00,Terminated,1.0,0.0
14234,30329.0,67245.0,NaN,Haulmer,Simple,Simple,60000.0,2021-03-07,2021-03-01,2023-03-01 00:00:00,Active,3.0,0.0


----